In [296]:
import numpy as np

n_stories = 4
n_cols = 4
dist_h = 8   # m
E = 2e11     # Pa (steel)
I = 2e-5     # m^4 (HEB200, weak axis)
A = 8e-3     # m^2
W = 2e-4     # m^3
# I = 4e-5     # m^4 (HEB240, weak axis)
# A = 1e-2     # m^2
# W = 3.3e-4   # m^3
# I = 8.5e-5   # m^4 (HEB300, weak axis)
# A = 1.5e-2   # m^2
# W = 5.7e-4   # m^3
h = 3        # m
m = 100*np.ones(n_stories)                # kg
k = n_cols*12*E*I/h**3*np.ones(n_stories) # kN/m
damping = 0.05                            # dim-less
z = np.arange(h, h*(n_stories+1), h)

m[0] = 3e5
m[1] = 2e5
m[2] = 1.5e5
m[3] = 1e5
# k[0] = 4e7
# k[1] = 3e7
# k[2] = 2e7
# k[3] = 1e7
m /= 5
print(m)

M = np.zeros((n_stories, n_stories))
K = np.zeros((n_stories, n_stories))
M[0,0] = m[0]
K[0,0] = k[0]
for i in range(1,n_stories):
  M[i,i] += m[i]
  K[i,i] += k[i]
  K[i-1,i-1] += k[i]
  K[i,i-1] += -k[i]
  K[i-1,i] += -k[i]

[60000. 40000. 30000. 20000.]


In [297]:
from scipy.linalg import eigh
eigvals, eigvecs = eigh(K, M)
eigvecs_n = [v/np.sqrt(v@M@v) for v in eigvecs.T] # normalization
eigvecs_n = np.array(eigvecs_n)      # casting into numpy array

In [298]:
frequencies = eigvals**0.5
periods = 2*np.pi/frequencies
print(f'frequency 1 : {frequencies[0]} rad/s')
print(f'period 1    : {periods[0]} s')
print(f'periods     : {periods}')

frequency 1 : 5.3829384655830115 rad/s
period 1    : 1.1672407825860354 s
periods     : [1.16724078 0.46486081 0.30318694 0.22481843]


In [299]:
d = np.ones(n_stories)
modal_mass = [(v@M@v) / (v@v) for v in eigvecs_n] # eigvecs.T
participation = [(v@M@d) / (v@M@v) for v in eigvecs_n] # eigvecs.T
eff_mass = [(v@M@d)**2 / (v@M@v) for v in eigvecs_n] # eigvecs.T
eff_mass_rel = eff_mass / np.trace(M)
print(f'Total mass      : {sum(eff_mass)}')
print(f'Effective  mass : {eff_mass}')
print(f'Participation   : {participation}')

Total mass      : 149999.99999999997
Effective  mass : [134099.03933303975, 14624.542860771791, 1260.743816041585, 15.673990146827656]
Participation   : [-366.19535678793056, 120.9319761716139, 35.50695447432214, -3.959039043357321]


In [300]:
g = 9.81            # m/s^2
dt = 0.02           # the time step considered by the input data
seism_scale = 0.01  # convert dm/s^2 to m/s^2
seism = np.loadtxt('AcMx1985.txt') * seism_scale
times = np.arange(len(seism))*dt

In [301]:
from scipy.integrate import solve_ivp

# NOTE: this method is not optimized because of the interpolation.
# Therefore, it is slower, although more accurate
def SystemDerivatives(t, y, frequency, damping):
    u = y[0]
    v = y[1]
    s = np.interp(t, times, seism)
    a = s - 2*damping*frequency*v - frequency**2*u
    return [v, a]

def MaxResponseInt(period, damping):
    frequency = 2*np.pi/period
    solution = solve_ivp(fun=SystemDerivatives, t_span=[0, times[-1]],
                         y0=[0,0], t_eval=times, args=[frequency, damping])
    a = frequency**2*solution.y[0]
    return max(np.absolute(a))

Sa = [MaxResponseInt(T,damping) for T in periods]
print('Sa : ', Sa)

Sa :  [0.41762629446404037, 3.9469130239109247, 2.4447923389622352, 1.8646082755697022]


In [302]:
u_max = []
v_max = []
a_max = []

for i in range(n_stories):
    w = frequencies[i]
    u_max.append(participation[i]*Sa[i]/w/w*eigvecs_n[i])
    v_max.append(participation[i]*Sa[i]/w*eigvecs_n[i])
    a_max.append(participation[i]*Sa[i]*eigvecs_n[i])

## Quadratic combination

$u_{CQC} = \sqrt{\sum_i u_i^2}$

In [303]:
print(u_max[0])

[0.00787546 0.01382549 0.0175221  0.01907676]


In [304]:
u_cqc = 0
for u_i in u_max:
    u_cqc += np.power(u_i,2)
u_cqc = np.sqrt(u_cqc)
u_cqc_rel = np.array(u_cqc) # creating a copy of the array
for i in range(1,n_stories):
    u_cqc_rel[i] = u_cqc[i] - u_cqc[i-1]
print(f'Complete quadratic combination')
print(f'absolute displacements : {u_cqc}')
print(f'relative displacements : {u_cqc_rel}')
print(3/500)

Complete quadratic combination
absolute displacements : [0.01131806 0.01433507 0.01809076 0.02121244]
relative displacements : [0.01131806 0.00301701 0.00375569 0.00312168]
0.006


## Base shear and overturning moment

$V_b = mS_a(\omega_1)$

$M_b = V_b \frac{\sum_i m_i z_i^2}{\sum_i m_i z_i}$

In [305]:
mass = np.trace(M)
Vb = mass*Sa[0]
Mb = Vb * np.dot(m, np.power(z,2)) / np.dot(m,z)
print(Vb)
print(Mb)

62643.94416960605
491047.6913940087


## Structure verification

In this example, we assume all the forces are equally distributed on all the columns. Each column must verify the following actions:

Shear: $V_{ED} = \frac{V_b}{n_{columns}}$

Bending moment: $M_{ED} = \frac{V_{ED}h}{2}$

Axial force: $N_{ED} = \frac{mg}{n_{columns}} + \frac{2M_b}{n_{columns}h_{dist}}$

In [306]:
Ved = Vb / n_cols
Med = Ved * h / 2
Ned = mass*10 / n_cols + 2*Mb/n_cols/dist_h
print(f'{Ved = } N')
print(f'{Med = } Nm')
print(f'{Ned = } N')

Ved = 15660.986042401513 N
Med = 23491.47906360227 Nm
Ned = 405690.48071212554 N


Resisting properties of a steel section

$V_{RD} = f_yA/\sqrt{3}$

$M_{RD} = f_yW$

$N_{RD} = f_yA$

In [307]:
fy = 3.6e8 # Pa
Vrd = fy*A/np.sqrt(3)
Mrd = fy*W
Nrd = fy*A
print(Vrd)
print(Mrd)
print(Nrd)

1662768.7752661223
72000.0
2880000.0


Verification

In [308]:
print(f'Ratio V : {Ved/Vrd}')
print(f'Ratio M : {Med/Mrd}')
print(f'Ratio N : {Ned/Nrd}')

Ratio V : 0.009418619278495298
Ratio M : 0.3262705425500315
Ratio N : 0.1408647502472658
